# Gradient Descent - Notes

![gradient_descent](assets/training-basics/gradient_descent.png)

(image: https://hackernoon.com/gradient-descent-aynk-7cbe95a778da)

Supplementary material that covers:
- why we need gradient descent
- how it is tuned
- various options for it

# Why do we need Gradient Descent?

- Computing what weights $w$ give the lowest cost is expensive
  - Lots of possible values of $w$
  - $w$ can have multiple dimensions (when you have multiple $x$ features)


# Why do we need Gradient Descent?

- Gradient Descent is an Approximation Algorithm
  - Finds the set of $w$ that can give "low enough" cost
  - Uses the direction of change in the cost function (the gradient)
  - The direction of change is relative to the values of $w$

# How do we tune Gradient Descent?

- Pick a learning rate that is not to fast or too slow
- Pick how the learning rate can be updated during training.
- Pick how long we train (number of epochs)

# Gradient Descent Algorithms

- Why do we have so many algorithms?
  - Because the approximation may not always find the minimum cost
  - Because we want to train faster

# Variations around Speed & Noise

- How fast we train
  - Gradient Descent: all samples (...slow...)
  - Stochastic Gradient Descent: random 1 sample (fast, but noisy)
  - Minibatch SGD: small batches (fast, less noisy)
  - Minibatch SGD + momentum: (fast, even less noisy)


When used:
- SGD is very common in machine learning
- Minibatch variations are common in deep learning (keras)

# Variations around Learning Rate

- How to pick the "best" learning rate
  - Make learning rate smaller as we train longer
    - Decaying learning rate
    - When used: common practice
  - Make learning rate slow down for steeper gradients, speed up for shallower gradients
    - RMSProp, AdaGrad, Adam, ...
    - When used: deep learning

# Gradient Descent Algorithms (pseudocode)

## Gradient Descent (aka Batch Gradient Descent)
```
params = random_initialize()

for i in range(n_epochs):

  # Entire data is trained each time
  params_grad = evaluate_gradient(loss_function, data, params)
  params = params - learning_rate * params_grad
  
return params
```

## Stochastic Gradient Descent

Difference with GD: picks a random sample to train

```
params = random_initialize()

for i in range(n_epochs):
  np.random.shuffle(data)

  # Gets one random example to train at a time
  for example in data:
    params_grad = evaluate_gradient(loss_function, example, params)
    params = params - learning_rate * params_grad

return params
```

sklearn: http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDRegressor.html

(Inspiration: http://ruder.io/optimizing-gradient-descent/index.html)


## Minibatch SGD

Difference with SGD: picks a batch (see `get_batches`) to train

```
params = random_initialize()

for i in range(n_epochs):
  np.random.shuffle(data)

  # Gets a batch of 50 to train at a time
  for batch in get_batches(data, batch_size=50):

    params_grad = evaluate_gradient(loss_function, batch, params)
    params = params - learning_rate * params_grad
    
return params
```

## Minibatch SGD + Momentum

Difference with Minibatch SGD: uses decaying moving average of previous gradients to update

```
params = random_initialize()

for i in range(n_epochs):
  np.random.shuffle(data)
  for batch in get_batches(data, batch_size=50):
    params_grad = evaluate_gradient(loss_function, batch, params)

    # Computes the decaying moving average of past gradients
    #   This becomes higher if previous gradient is same direction
    #   Lower if previous gradient is opposite direction
    velocity = momentum * velocity - learning_rate * params_grad

    # Uses the moving average to update
    params = params + velocity
    
return params
```

Keras implementation:https://github.com/keras-team/keras/blob/master/keras/optimizers.py#L146

### Nesterov Accelerated Gradient Descent (NAG)

- A variant of Minibatch SGD + Momentum
- Look ahead by calculating gradient w.r.t. approximate future value of $\theta$ (i.e. $\nabla_{\theta}L(\theta - \gamma v)$)

$$v \leftarrow \gamma v + \epsilon \nabla_{\theta}L(\theta - \gamma v) $$

$$\theta \leftarrow \theta - v$$

- Allows gradient updates to slow down if the future slope is going upwards
- Popular with deep learning (e.g. Baidu DeepSpeech)

Keras implementation: https://github.com/keras-team/keras/blob/master/keras/optimizers.py#L146

### Nesterov Accelerated Gradient

Difference with Minibatch SGD with Momentum: uses the next possible params to compute gradient.

```
params = random_initialize()

for i in range(n_epochs):
  np.random.shuffle(data)
  for batch in get_batches(data, batch_size=50):
  
    # Computes the future params
    lookahead_params = params - momentum * velocity

    # Get the gradient of the future params
    params_grad = evaluate_gradient(loss_function, batch, lookahead_params)

    velocity = momentum * velocity + learning_rate * params_grad
    params = params - velocity
    
return params
```

### Adaptive Learning Rate Strategies

- Learning rate will control the amount of gradient update
  - Large learning rate: risk overshoot and not converge
  - Small learning rate: too slow
  - Ideal: start large(r), then reduce as we get closer to minima
- Strategies
  - Constant learning rate
  - Time-based or step-based decay
  - Adaptive learning rate
    - RMSProp
    - AdaGrad
    - Adam
- What works best depends on your domain (true for **any** optimization)

### RMSProp

- Divide learning rate by exponentially decaying average of past squared gradients
   - Steeper gradients: smaller learning rate
- Usually good for RNNs

$$E[g^2]_t = 0.9E[g^2]_{t-1} + 0.1g_t^2 $$

$$\theta \leftarrow \theta - \frac{\epsilon}{\sqrt{E[g^2]_t + c}} g$$

$c$ = small constant to avoid division by zero

Keras implementation:
https://github.com/keras-team/keras/blob/master/keras/optimizers.py#L209

### AdaGrad

- Adaptive, per-parameter learning rate
- Designed for sparse features
  - more frequent features: smaller learning rate (e.g. common words in a document)
  - more rare features: higher learning rate (e.g. rare words in a document)
- learning rate gets updated as parameter gets updated

$$\theta \leftarrow \theta - \frac{\epsilon}{\sqrt{G_t + c}} g$$

$G_t \in I\!R^{d\times d} $ = diagonal matrix
- sum of squares of past gradients
- 1 diagonal element per parameter

$c$ = small constant to avoid division by zero

Keras implementation:
https://github.com/keras-team/keras/blob/master/keras/optimizers.py#L276

### Adam (Adaptive Moment Estimation)
- Requires less fine-tuning (easier to use)
- Estimated first moment (mean): $m \leftarrow \beta_1 m + (1-\beta_1)g$
  - Averages out the gradient used for update
- Estimated second moment (uncentered variance): $v \leftarrow \beta_2 v + (1-\beta_2)g^2$
  - Smooths out the learning rate if gradient is high

$$\theta \leftarrow \theta - \frac{\epsilon}{\sqrt{\hat{v}} + c}\hat{m}$$
- $\hat{v} = \frac{v}{1-\beta_2}$, $\hat{m} = \frac{m}{1-\beta_1}$ to avoid zero bias
- $\beta_1$ and $\beta_2$ usually 0.9 and 0.999
- $c$ = small constant to avoid division by zero

Keras implementation:
https://github.com/keras-team/keras/blob/master/keras/optimizers.py#L422

# Mean Squared Error - Gradients

The following example shows how the Partial Derivative of the Mean Square Error function is computed.

The model has 2 weights: $\theta_0$ and $\theta_1$ (alias for $w_0$ and $w_1$)


![gradient MSE](assets/training-basics/ThetaZeroDerivation.png)

(image: http://mccormickml.com/2014/03/04/gradient-descent-derivation/)

![thetazeroderivativeoferror1.png](assets/training-basics/thetazeroderivativeoferror1.png)

![ThetaOneDerivativeOfError.png](assets/training-basics/ThetaOneDerivativeOfError.png)

(image: http://mccormickml.com/2014/03/04/gradient-descent-derivation/)

![gradientdescentofmsetable](assets/training-basics/gradientdescentofmsetable.png)

(image: http://mccormickml.com/2014/03/04/gradient-descent-derivation/)